In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
import io

In [ ]:
from google.colab import files
upload = files.upload()

Saving heart.csv to heart.csv


In [ ]:
df = pd.read_csv(io.BytesIO(upload['heart.csv']))

In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
X= df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2 , random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier() # raw without any paramter

In [ ]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred  = rf.predict(X_test)

In [ ]:
print("Accuracy of random forest: ",accuracy_score(y_test,y_pred))

Accuracy of random forest:  0.8360655737704918


In [ ]:
clf = RandomForestClassifier()

In [ ]:
np.mean(cross_val_score(clf , X,y, cv = 5, scoring ='accuracy'))

0.8149180327868853

# Now include some feature and tune the parameter

In [ ]:
rtf = RandomForestClassifier(
    n_estimators= 50,
    max_samples = 0.5,
    max_features = 0.75,
    bootstrap = True
)

In [ ]:
rtf.fit(X_train,y_train)

RandomForestClassifier(max_features=0.75, max_samples=0.5, n_estimators=50)

In [ ]:
y_pred = rtf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred) # as we can see accuracy has increased

0.8524590163934426

# Hyper Parameter tuning with GridSearchCV

In [ ]:
param ={
    'n_estimators':[50,100,150],
    'max_depth':[None,4,10],
    'max_features':[0.5,0.75,1.0],
    'max_samples': [0.5,0.75]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(
    estimator= RandomForestClassifier(),
    param_grid= param,
    cv =5,
    verbose= 2,
    n_jobs= -1
)

In [ ]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 4, 10],
                         'max_features': [0.5, 0.75, 1.0],
                         'max_samples': [0.5, 0.75],
                         'n_estimators': [50, 100, 150]},
             verbose=2)

In [ ]:
search.best_estimator_

RandomForestClassifier(max_depth=10, max_features=0.75, max_samples=0.5)

In [ ]:
search.best_params_

{'max_depth': 10,
 'max_features': 0.75,
 'max_samples': 0.5,
 'n_estimators': 100}

In [ ]:
search.best_score_

0.8262755102040817

# RandomSearchCV to optimize the hyper parameter

## It is used in larger and bigger data becaise it takes the random sample and it is faster than GridSearchCv

In [ ]:
param ={
    'n_estimators':[50,100,150],
    'max_depth':[None,4,10],
    'min_samples_split':[2,4],
    'max_features':[0.5,0.75,1.0],
    'max_samples': [None,0.5,0.75],
    'min_samples_leaf':[1,2]

}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rnsh = RandomizedSearchCV(
    estimator= RandomForestClassifier(),
    param_distributions= param,
    cv =5,
    verbose= 2,
    n_jobs= -1
)

In [ ]:
rnsh.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [None, 4, 10],
                                        'max_features': [0.5, 0.75, 1.0],
                                        'max_samples': [None, 0.5, 0.75],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 4],
                                        'n_estimators': [50, 100, 150]},
                   verbose=2)

In [ ]:
rnsh.best_params_

{'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_samples': 0.5,
 'max_features': 0.75,
 'max_depth': 10}

In [ ]:
rnsh.best_score_

0.8098639455782314

In [ ]:
rnsh.best_estimator_

RandomForestClassifier(max_depth=10, max_features=0.75, max_samples=0.5,
                       n_estimators=150)

## OBB Score

In [53]:
ob_rf = RandomForestClassifier(
    max_depth = 10,
    max_features =  0.75,
    max_samples = 0.5,
    n_estimators = 100,
    oob_score = True,
)

In [55]:
ob_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, max_features=0.75, max_samples=0.5,
                       oob_score=True)

In [58]:
ob_rf.oob_score_  # accuracy on unseen and untrainde data out of bag samples

0.8016528925619835